# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

[INFO] 1451311092 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_FILE to /Users/Baran/anaconda/lib/python2.7/site-packages/certifi/cacert.pem
1451311092 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_DIR to 
This non-commercial license of GraphLab Create is assigned to bbuluttekin@hotmail.com and will expire on September 21, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-4159 - Server binary: /Users/Baran/anaconda/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1451311092.log
[INFO] GraphLab Server Version: 1.7.1


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 21613
PROGRESS: Number of features          : 17
PROGRESS: Number of unpacked features : 17
PROGRESS: Number of coefficients    : 18
PROGRESS: Starting Accelerated Gradient (FISTA)
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: Tuning step size. First iteration could take longer than subsequent iterations.
PROGRESS: | 1         | 2        | 0.000002  | 1.493978     | 6962915.603493     | 426631.749026 |
PROGRESS: | 2         | 3        | 0.000002  | 1.534606     | 6843144.200219     | 392488.929838 |
PROGRESS: | 3         | 4      

Find what features had non-zero weight.

In [11]:
count = 0
for items in model_all.coefficients['value']:
    if items != 0.0:
        count += 1
print count

6


In [22]:
model_all.coefficients[model_all.coefficients['name'] == "sqft_living"]

name,index,value
sqft_living,None,24.4207209824


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [23]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [24]:
import numpy as np

In [25]:
RSS_dict = {}
l1_penalty_list = np.logspace(1, 7, num = 13)
for penalty in l1_penalty_list:
    model = graphlab.linear_regression.create(training, target = "price", features= all_features,
                                             l1_penalty = penalty, l2_penalty = 0, verbose= False,
                                             validation_set = None)
    predictions = model.predict(validation)
    RSS = ((validation["price"] - predictions)**2).sum()
    RSS_dict[RSS] = penalty
print RSS_dict[np.min(RSS_dict.keys())]

10.0


In [27]:
RSS_dict

{625766285142459.9: 10.0,
 625766285362394.1: 31.622776601683793,
 625766286057885.0: 100.0,
 625766288257224.6: 316.22776601683796,
 625766295212186.8: 1000.0,
 625766317206080.5: 3162.2776601683795,
 625766386760658.1: 10000.0,
 625766606749278.5: 31622.776601683792,
 625767302791634.1: 100000.0,
 625769507643886.2: 316227.76601683791,
 625776517727024.0: 1000000.0,
 625799062845467.0: 3162277.6601683795,
 625883719085425.2: 10000000.0}

In [29]:
m_win = graphlab.linear_regression.create(training, target = "price", features= all_features,
                                          l1_penalty = 10.0, l2_penalty = 0, verbose= False,
                                          validation_set = None)
win_count = 0
for items in m_win.coefficients['value']:
    if items != 0.0:
        win_count += 1
print win_count

18


*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [30]:
RSS_test = ((testing["price"] - m_win.predict(testing))**2).sum()
RSS_test

156983602381664.2

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [31]:
m_win.coefficients

name,index,value
(intercept),None,18993.4272128
bedrooms,None,7936.96767903
bedrooms_square,None,936.993368193
bathrooms,None,25409.5889341
sqft_living,None,39.1151363797
sqft_living_sqrt,None,1124.65021281
sqft_lot,None,0.00348361822299
sqft_lot_sqrt,None,148.258391011
floors,None,21204.335467
floors_square,None,12915.5243361


# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [32]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [33]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [34]:
esp = {}
for pen in l1_penalty_values:
    m = graphlab.linear_regression.create(training, target= "price", features= all_features,
                                         l1_penalty= pen, l2_penalty= 0, verbose= False,
                                         validation_set= None)
    non_zero = m["coefficients"]["value"].nnz()
    esp[non_zero] = pen
esp

{1: 10000000000.0,
 3: 6158482110.6602545,
 5: 4832930238.5717525,
 6: 3792690190.7322536,
 10: 2976351441.6313133,
 12: 2335721469.0901213,
 13: 1832980710.8324375,
 15: 1438449888.2876658,
 16: 885866790.41008317,
 17: 695192796.17755914,
 18: 206913808.11147901}

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [44]:
max_nonzero = [esp[nonz] for nonz in esp.keys() if nonz > max_nonzeros]
f_max_nonzero = [esp[nonz] for nonz in esp.keys() if nonz < max_nonzeros]

In [45]:
l1_penalty_min = np.max([esp[nonz] for nonz in esp.keys() if nonz > max_nonzeros])
l1_penalty_max = np.min([esp[nonz] for nonz in esp.keys() if nonz < max_nonzeros])

In [46]:
print l1_penalty_min, l1_penalty_max

2976351441.63 3792690190.73


***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [47]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [57]:
RSS_last = {}
for p in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target = "price", features= all_features,
                                             l1_penalty = p, l2_penalty = 0, verbose= False,
                                             validation_set = None)
    sp = model["coefficients"]["value"].nnz() == max_nonzeros
    predictions = model.predict(validation)
    RSS = ((validation["price"] - predictions)**2).sum()
    RSS_last[RSS] = [p, sp]
print RSS_last[np.min(RSS_last.keys())][0]

2976351441.63


In [58]:
RSS_last

{966925692362084.5: [2976351441.6313133, False],
 974019450084556.1: [3019316638.9524155, False],
 981188367942452.8: [3062281836.2735176, False],
 989328342459474.0: [3105247033.5946198, False],
 998783211265891.2: [3148212230.9157219, False],
 1008477167020094.0: [3191177428.236824, False],
 1018298780553819.8: [3234142625.5579262, False],
 1028247992205977.2: [3277107822.8790283, False],
 1034616909232828.1: [3320073020.2001305, False],
 1038554735941040.8: [3363038217.5212326, False],
 1043237237871703.0: [3406003414.8423347, False],
 1046937488751711.1: [3448968612.1634369, True],
 1051147625612860.9: [3491933809.484539, True],
 1055992735342999.1: [3534899006.8056412, True],
 1060799531763287.8: [3577864204.1267428, True],
 1065707689498230.1: [3620829401.447845, False],
 1069464335425586.5: [3663794598.7689471, False],
 1073504549585599.6: [3706759796.0900493, False],
 1077632775581416.0: [3749724993.4111514, False],
 1081867592324110.6: [3792690190.7322536, False]}

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [53]:
f_mod = graphlab.linear_regression.create(training, target = "price", features= all_features,
                                          l1_penalty = 3620829401.447845, l2_penalty = 0, verbose= False,
                                          validation_set = None)
f_mod["coefficients"]

name,index,value
(intercept),None,231334.909221
bedrooms,None,0.0
bedrooms_square,None,0.0
bathrooms,None,14892.9994878
sqft_living,None,31.5287758428
sqft_living_sqrt,None,642.87941674
sqft_lot,None,0.0
sqft_lot_sqrt,None,0.0
floors,None,0.0
floors_square,None,0.0


In [56]:
f_mod.coefficients[f_mod.coefficients["name"] == "floors"]

name,index,value
floors,None,0.0
